In [9]:
import pandas as pd

In [10]:
# Read the CSV file into a DataFrame
train = pd.read_csv('/Users/arup/Documents/BAS/train.csv')
test = pd.read_csv('/Users/arup/Documents/BAS/test.csv')

In [11]:
train

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [12]:
# Identify the columns you want to delete
columns_to_delete = ['textID', 'selected_text']

# Use the drop method to remove the specified columns
train = train.drop(columns=columns_to_delete)

# Save the modified DataFrame to a new file
train.to_csv('modified_train.csv', index=False)

In [13]:
train

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...
27476,wish we could come see u on Denver husband l...,negative
27477,I`ve wondered about rake to. The client has ...,negative
27478,Yay good for both of you. Enjoy the break - y...,positive
27479,But it was worth it ****.,positive


In [14]:
print(train.dtypes)

text         object
sentiment    object
dtype: object


In [15]:
test

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive
...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative
3530,416863ce47,All alone in this old house again. Thanks for...,positive
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive


In [16]:
# Identify the columns you want to delete
columns_to_delete = ['textID']

# Use the drop method to remove the specified columns
test = test.drop(columns=columns_to_delete)

# Save the modified DataFrame to a new file
test.to_csv('modified_test.csv', index=False)

In [17]:
test

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive
...,...,...
3529,"its at 3 am, im very tired but i can`t sleep ...",negative
3530,All alone in this old house again. Thanks for...,positive
3531,I know what you mean. My little dog is sinkin...,negative
3532,_sutra what is your next youtube video gonna b...,positive


In [18]:
import nltk
import numpy as np
from pyswarm import pso
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [19]:
# Define a basic tokenization function
def tokenize(text):
    return nltk.word_tokenize(text)

# Tokenization and stop word removal
stop_words = set(nltk.corpus.stopwords.words('english'))

In [48]:
# Modify calculate_sentiment_score to accept a list
def calculate_sentiment_score(review, polarity_values):
    sentiment_score = 0
    for word in nltk.word_tokenize(review.lower()):
        if word in stop_words:
            word_index = list(stop_words).index(word)
            sentiment_score += polarity_values.get(word_index, 0)
        else:
            # Handle the case when the word is not in stop_words
            sentiment_score += 0  # You can choose another default value
    return sentiment_score

In [49]:
# Modify the objective function to handle 3 classes
def objective_function(polarity_values, *args):
    data, labels = args
    sentiment_scores = []

    for review, label in zip(data, labels):
        sentiment_score = 0.0
        for word in nltk.word_tokenize(" ".join(review)):
            if word in stop_words:
                sentiment_score += polarity_values[list(stop_words).index(word)]
        sentiment_scores.append(sentiment_score)

    # Convert labels to numerical values
    label_mapping = {'positive': 1, 'negative': -1}
    numerical_labels = np.array([label_mapping[label] for label in labels])

    # Evaluate accuracy
    predicted_labels = np.array(sentiment_scores)
    accuracy = np.sum(np.sign(predicted_labels) == numerical_labels) / len(numerical_labels)

    # Maximize the accuracy
    return -accuracy

In [50]:
# Modify lexicon_labeling_pso to handle 3 classes
def lexicon_labeling_pso(data, labels):
    lb = np.zeros(len(stop_words))  # Lower bounds for polarity values
    ub = np.ones(len(stop_words))  # Upper bounds for polarity values

    # Initialize polarity values
    initial_polarity_values = np.array([1.0] * len(stop_words)).reshape(-1)

    # Run PSO optimization
    best_polarity_values, _ = pso(objective_function, lb, ub, args=(data, labels), swarmsize=10, maxiter=100, minstep=0.01, minfunc=0.01, debug=True)

    # Convert the polarity values to a dictionary
    polarity_dict = dict(zip(list(stop_words), best_polarity_values))

    # Return the best-polarity lexicon as a dictionary
    return polarity_dict

In [51]:
# Tokenize your data (data should be defined somewhere in your code)
tokenized_data = [" ".join(tokenize(review[0])) for review in data]

# Lexicon labeled by PSO
pso_labeled_lexicon = lexicon_labeling_pso(tokenized_data, [label for _, label in data])

# Extract features using lexicon-based sentiment analysis
features = [[calculate_sentiment_score(review, pso_labeled_lexicon)] for review in tokenized_data]

# Convert labels to numerical values
label_mapping = {'positive': 1, 'negative': -1}
labels = np.array([label_mapping[label] for _, label in data])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

No constraints given.
Best after iteration 1: [7.32981276e-01 4.71133605e-01 8.30578801e-01 6.88052965e-01
 7.28060798e-01 2.35794433e-01 7.21344629e-01 6.66576382e-01
 4.02747757e-01 3.23951976e-01 9.00594106e-01 2.62207646e-01
 3.76436830e-01 3.39071041e-01 6.76018358e-01 3.33317052e-01
 6.67073634e-01 5.23385012e-01 2.59330831e-01 7.34438012e-01
 1.08790631e-01 4.56017511e-01 2.02416286e-01 7.22871558e-01
 7.59900238e-01 8.98532404e-02 7.63060487e-01 7.08394654e-01
 5.52277057e-01 1.82846413e-01 2.40904210e-02 3.35220316e-01
 1.79195065e-01 5.55379857e-01 3.32568870e-01 2.73597741e-01
 9.61084500e-01 2.17520999e-01 1.33062705e-01 5.74069446e-01
 1.15968071e-01 3.35749209e-01 5.57371207e-01 3.57985348e-01
 3.17167369e-01 8.69595880e-01 6.11793085e-01 5.01247293e-01
 6.29075909e-01 2.04156069e-01 1.65170502e-01 5.81597867e-01
 6.88061259e-01 8.38524365e-01 7.09289755e-01 6.03487068e-03
 8.10811138e-01 7.23941424e-01 6.84935684e-01 4.53782571e-01
 1.68828592e-01 4.79927056e-01 3.257295

In [52]:
# Train a Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [53]:
# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_mapping.keys()))

Classification Report:
              precision    recall  f1-score   support

    positive       0.50      1.00      0.67         5
    negative       0.00      0.00      0.00         5

    accuracy                           0.50        10
   macro avg       0.25      0.50      0.33        10
weighted avg       0.25      0.50      0.33        10



/Users/arup/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arup/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arup/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
